In [1]:
import numpy as np
import math
import scipy

from matplotlib import pyplot as plt
import matplotlib.cm as cm
plt.rcParams['figure.figsize'] = [10, 10]
plt.rc('font', size=20)

import torch
import torch.optim as optim
import torch.nn as nn
import torch.nn.functional as F
import torch.nn.init as init

import transformers
from datasets import load_dataset, DatasetDict
from transformers import AutoTokenizer
from torch.utils.data import Dataset, DataLoader

from functools import partial

# from collections import Counter

import os
import argparse
import time
from tqdm import tqdm # Loading bar
print('Done.')

Done.


In [2]:
from utils import complex_conj_transpose, batched_complex_conj_transpose, complex_exp, complex_exp_v2, complex_hadamard, complex_matmul, complex_division
from utils import batched_complex_conj_transpose, batched_complex_hadamard, batched_complex_matmul, batched_complex_division
from utils import batched_complex_exp, batched_complex_hadamard_full, batched_complex_matmul_full
print('Done.')

Done.


In [3]:
from dynamics import stochastic_LTI, DynamicSim
from dynamics import construct_mapping
from dynamics import get_nth_measurement, get_random_measurements
from dynamics import linear_spiral, linear_spiral_3D, Lorenz, rand_coupling_matrix, Van_der_Pol_osc
print('Done.')

Done.


In [4]:
from precision_attention import compute_residuals, compute_kernel_v1, compute_estimates_and_residuals_vectorized, get_time_diffs, compute_neg_kernel, clamp_exponent_arg
from precision_attention import compute_kernel, batched_compute_estimates_and_residuals_vectorized, compute_estimates_and_residuals_irregular_times, compute_nu
from precision_attention import compute_precision_v1
# from precision_attention import precise_attn, precise_attn_with_correction, precise_attn_full
from precision_attention import compute_precision, compute_precision_tanh
print('Done.')

Done.


In [5]:
from model import compute_lambda_h
from model import init_complex_matrix, build_nearly_identity, initialize_to_correct_model
from model import init_weight_masks, apply_weight_masks
from model import Complex_MSE_Loss, Batched_Complex_MSE_Loss, inverse_penalty
from model import BatchedPrecisionAttentionBlock
from model import HadamardLayer, TemporalNorm, TemporalWhiteningLayer
from model import PrecisionNet_1layer, PrecisionNet
print('Done.')

Done.


In [37]:
parser = argparse.ArgumentParser('DA')
parser.add_argument('--gpu', type=int, default=0) # (Default: 0)
args = parser.parse_args(args=[])
args.device = torch.device('cuda:' + str(args.gpu) if torch.cuda.is_available() else 'cpu')
print(args.device)

torch.manual_seed(2025)
np.random.seed(2025)

cuda:0


In [6]:
# Visualize data

import pandas as pd

local_data_dir = r"C:\Users\Pracioppo\Desktop\Peter DynAttn Proj\data\wikitext"

# Replace 'your_file.parquet' with the actual path to your Parquet file

try:
    # Load the Parquet file into a Pandas DataFrame
    df = pd.read_parquet(local_data_dir)

    print("--- DataFrame Head (First 5 rows) ---")
    print(df.head())

    print("\n--- DataFrame Info (Columns, Non-Null Counts, Dtypes) ---")
    df.info()

    print("\n--- DataFrame Description (Statistical Summary) ---")
    print(df.describe()) # For numerical columns

    print(f"\n--- DataFrame Shape (Rows, Columns) ---")
    print(f"Shape: {df.shape}")

    print(f"\n--- DataFrame Columns ---")
    print(df.columns.tolist())

except FileNotFoundError:
    print(f"Error: The file '{file_path}' was not found.")
except Exception as e:
    print(f"An error occurred: {e}")

--- DataFrame Head (First 5 rows) ---
                                                text
0                                                   
1                              = Robert Boulter = \n
2                                                   
3   Robert Boulter is an English film , televisio...
4   In 2006 , Boulter starred alongside Whishaw i...

--- DataFrame Info (Columns, Non-Null Counts, Dtypes) ---
<class 'pandas.core.frame.DataFrame'>
RangeIndex: 44836 entries, 0 to 44835
Data columns (total 1 columns):
 #   Column  Non-Null Count  Dtype 
---  ------  --------------  ----- 
 0   text    44836 non-null  object
dtypes: object(1)
memory usage: 350.4+ KB

--- DataFrame Description (Statistical Summary) ---
         text
count   44836
unique  26538
top          
freq    15717

--- DataFrame Shape (Rows, Columns) ---
Shape: (44836, 1)

--- DataFrame Columns ---
['text']


In [7]:
# # --- Define the local path where you saved the files ---
# # IMPORTANT: Replace this with the actual path on your computer!
# local_data_dir = r"C:\Users\Pracioppo\Desktop\Peter DynAttn Proj\data\wikitext"

# # Load the Dataset
# local_data_files = {
#     'train': f"{local_data_dir}/train-00000-of-00001.parquet",
#     'validation': f"{local_data_dir}/validation-00000-of-00001.parquet",
#     'test': f"{local_data_dir}/test-00000-of-00001.parquet"
# }

# try:
#     raw_dataset = load_dataset('parquet', data_files=local_data_files)
#     print("Dataset loaded successfully from local files!")
#     print(raw_dataset)
# except Exception as e:
#     print(f"Error loading local files: {e}")
#     print("Please ensure the 'local_data_dir' path is correct and the files are present.")

Using custom data configuration default-95672dcb03904034


Extracting data files:   0%|          | 0/3 [00:00<?, ?it/s]

0 tables [00:00, ? tables/s]

0 tables [00:00, ? tables/s]

0 tables [00:00, ? tables/s]

Dataset parquet downloaded and prepared to file://C:/Users/Pracioppo/.cache/huggingface/datasets/parquet/default-95672dcb03904034/0.0.0/2a3b91fbd88a2c90d1dbbb32b460cf621d31bd5b05b934492fdef7d8d6f236ec. Subsequent calls will reuse this data.
Error loading local files: Loading a dataset cached in a LocalFileSystem is not supported.
Please ensure the 'local_data_dir' path is correct and the files are present.


In [10]:
import pandas as pd # Make sure pandas is imported
from datasets import Dataset, DatasetDict # Ensure Dataset and DatasetDict are imported

# --- Define the local path where you saved the files ---
# IMPORTANT: Replace this with the actual path on your computer!
local_data_dir = r"C:\Users\Pracioppo\Desktop\Peter DynAttn Proj\data\wikitext"

# Define the full paths to your parquet files
train_file = f"{local_data_dir}/train-00000-of-00001.parquet"
validation_file = f"{local_data_dir}/validation-00000-of-00001.parquet"
test_file = f"{local_data_dir}/test-00000-of-00001.parquet"

try:
    print("Attempting to load datasets via Pandas workaround...")

    # 1. Load each parquet file into a Pandas DataFrame
    train_df = pd.read_parquet(train_file)
    validation_df = pd.read_parquet(validation_file)
    test_df = pd.read_parquet(test_file)

    print("Parquet files loaded into Pandas DataFrames successfully.")

    # 2. Convert Pandas DataFrames to Hugging Face Dataset objects
    train_dataset = Dataset.from_pandas(train_df)
    validation_dataset = Dataset.from_pandas(validation_df)
    test_dataset = Dataset.from_pandas(test_df)

    print("Pandas DataFrames converted to Hugging Face Dataset objects.")

    # 3. Create a DatasetDict from these Dataset objects
    raw_dataset = DatasetDict({
        'train': train_dataset,
        'validation': validation_dataset,
        'test': test_dataset
    })

    print("\nDataset loaded successfully via Pandas workaround!")
    print(raw_dataset)

except FileNotFoundError:
    print(f"Error: One or more local files not found in '{local_data_dir}'.")
    print("Please ensure the 'local_data_dir' path is correct and the files (train-*.parquet, validation-*.parquet, test-*.parquet) are present.")
except Exception as e:
    print(f"An unexpected error occurred during Pandas loading: {e}")
    print("Please ensure you have pandas and pyarrow installed:")
    print("pip install pandas pyarrow")


Attempting to load datasets via Pandas workaround...
Parquet files loaded into Pandas DataFrames successfully.
Pandas DataFrames converted to Hugging Face Dataset objects.

Dataset loaded successfully via Pandas workaround!
DatasetDict({
    train: Dataset({
        features: ['text'],
        num_rows: 36718
    })
    validation: Dataset({
        features: ['text'],
        num_rows: 3760
    })
    test: Dataset({
        features: ['text'],
        num_rows: 4358
    })
})


In [11]:
# import re # Import regex for more flexible text cleaning
# from langdetect import detect, DetectorFactory
# DetectorFactory.seed = 0 # for reproducibility

def clean_wikitext_examples(examples):
    cleaned_texts = []
    for text in examples["text"]:
        # Step 1: Strip leading/trailing whitespace (including newlines)
        stripped_text = text.strip()

        # Step 2: Filter out empty strings and Wikipedia-style headings
        if not stripped_text or \
           (stripped_text.startswith('=') and stripped_text.endswith('=') and len(stripped_text) < 100) or \
           len(stripped_text) < 100: # Filter very short lines that are likely not useful sentences
            continue # Skip this example if it matches criteria

        # Step 3: Remove common Wikitext artifacts like '@-@', '= =' using string.replace()
        # It's good to replace these with a single space to avoid merging words.
        cleaned_text = stripped_text.replace(" @-@ ", " ") # Replace " @-@ " with a space
        cleaned_text = cleaned_text.replace(" = = ", " ")  # Replace " = = " with a space

        # Step 4: Normalize spaces (replace multiple spaces with a single space)
        # This is typically done by splitting by space and joining back,
        # which also handles leading/trailing spaces if not already stripped.
        cleaned_text = ' '.join(cleaned_text.split())

        # Step 5: (Optional, more advanced) Language filtering for non-English content.
        # This still requires an external library (e.g., `pip install langdetect`).
        # from langdetect import detect, DetectorFactory
        # DetectorFactory.seed = 0 # for reproducibility
        # try:
        #     if detect(cleaned_text) != 'en':
        #         continue # Skip non-English text
        # except Exception: # Handles cases where text is too short for reliable detection
        #     pass

        if cleaned_text: # Ensure it's not empty after all cleaning steps
            cleaned_texts.append(cleaned_text)

    return {"text": cleaned_texts}

In [13]:
cleaned_dataset = raw_dataset.map(
    clean_wikitext_examples,
    batched=True,
    remove_columns=["text"], # The 'text' column will be replaced by the cleaned one
)

  0%|          | 0/37 [00:00<?, ?ba/s]

  0%|          | 0/4 [00:00<?, ?ba/s]

  0%|          | 0/5 [00:00<?, ?ba/s]

In [14]:
print(f"Original train split size: {len(raw_dataset['train'])}")
print(f"Cleaned train split size: {len(cleaned_dataset['train'])}")
print(f"Example cleaned text from train split (first 500 chars):")
print(cleaned_dataset['train'][0]['text'][:500]) # This should now be actual content
print(cleaned_dataset['train'][1]['text'][:500])

Original train split size: 36718
Cleaned train split size: 15283
Example cleaned text from train split (first 500 chars):
Senjō no Valkyria 3 : Unrecorded Chronicles ( Japanese : 戦場のヴァルキュリア3 , lit . Valkyria of the Battlefield 3 ) , commonly referred to as Valkyria Chronicles III outside Japan , is a tactical role playing video game developed by Sega and Media.Vision for the PlayStation Portable . Released in January 2011 in Japan , it is the third game in the Valkyria series . Employing the same fusion of tactical and real time gameplay as its predecessors , the story runs parallel to the first game and follows th
The game began development in 2010 , carrying over a large portion of the work done on Valkyria Chronicles II . While it retained the standard features of the series , it also underwent multiple adjustments , such as making the game more forgiving for series newcomers . Character designer Raita Honjou and composer Hitoshi Sakimoto both returned from previous entries , along 

### From txt

In [ ]:
# import string
# import os # For creating/cleaning up the dummy file

# """
# Loads a text file, removes words that are entirely in uppercase,
# and breaks the processed text into sections of 500 words.

# Args:
#     file_path (str): The path to the input text file.

# Returns:
#     list: A list of strings, where each string is a section of up to 500 words.
#           Returns an empty list if the file is not found or an error occurs.
# """

# txt_dir = r"C:\Users\Pracioppo\Desktop\Peter DynAttn Proj\data\poets"

# file_path = txt_dir + "\\" + "milton" + ".txt"

# file_path

# # 1. Load the text file
# try:
#     with open(file_path, 'r', encoding='utf-8') as f:
#         full_text = f.read()
# except FileNotFoundError:
#     print(f"Error: The file '{file_path}' was not found.")
# except Exception as e:
#     print(f"An error occurred while reading the file: {e}")

# # 2. Remove all words that are entirely in uppercase
# # Split the full text into individual words based on whitespace
# words = full_text.split()
# filtered_words = []
# for word in words:
#     # Create a version of the word to check for all-caps property.
#     # This involves stripping punctuation from its ends, so "WORD!" becomes "WORD".
#     word_for_check = word.strip(string.punctuation)

#     # Check if the stripped word consists only of alphabetic characters
#     # AND if all those alphabetic characters are uppercase.
#     # This prevents numbers, symbols, or mixed-case words from being
#     # incorrectly removed (e.g., "123ABC" or "Python").
#     if word_for_check.isalpha() and word_for_check.isupper():
#         # If it's an all-caps alphabetic word, we skip it (don't add to filtered_words)
#         continue
#     else:
#         # Otherwise, keep the original word (with its punctuation)
#         filtered_words.append(word)

# # Join the filtered words back into a single string
# cleaned_text = ' '.join(filtered_words)

# # 3. Break the cleaned text into sections of 500 words
# # First, split the cleaned text back into a list of words.
# cleaned_words_list = cleaned_text.split()

# # sections = []
# # # Iterate through the list, taking chunks of 500 words
# # for i in range(0, len(cleaned_words_list), 500):
# #     section_words = cleaned_words_list[i : i + 500] # Slice the list
# #     sections.append(' '.join(section_words)) # Join the words in the section back into a string

# # len(sections)

### Tokenizer

In [15]:
# Load tokenizer

# from transformers import BertTokenizer, BertModel
# tokenizer = AutoTokenizer.from_pretrained("bert-base-uncased")

# local_bert_dir = r"C:\Users\Pracioppo\Desktop\Peter DynAttn Proj\data\local_bert_dir"
# tokenizer = BertTokenizer.from_pretrained(local_bert_dir)
# print("Tokenizer loaded!")

local_bert_mini_dir = r"C:\Users\Pracioppo\Desktop\Peter DynAttn Proj\data\prajjwal1_bert_mini"
tokenizer = AutoTokenizer.from_pretrained(local_bert_mini_dir)
print("Tokenizer loaded!")

# Ensure tokenizer has a pad_token if your model or DataCollator needs it
if tokenizer.pad_token is None:
    tokenizer.add_special_tokens({'pad_token': '[PAD]'})
    print("Added padding.")

Tokenizer loaded!


In [16]:
def tokenize_function(examples):
    # This function uses the 'tokenizer' object defined globally above
    return tokenizer(examples["text"]) # No truncation/max_length here for LM concatenation

In [17]:
# Apply Tokenization (Corrected map call)
print("\nTokenizing cleaned dataset...")
tokenized_dataset = cleaned_dataset.map(
    tokenize_function,
    batched=True,
    remove_columns=["text"], # This removes the original 'text' column after tokenization
)


Tokenizing cleaned dataset...


  0%|          | 0/16 [00:00<?, ?ba/s]

  0%|          | 0/2 [00:00<?, ?ba/s]

  0%|          | 0/2 [00:00<?, ?ba/s]

In [18]:
# Define Block Size for Language Modeling
block_size = 128

# --- 8. Define Grouping Function for Language Modeling ---
def group_texts(examples):
    concatenated_examples = {k: sum(examples[k], []) for k in examples.keys()}
    total_length = len(concatenated_examples[list(examples.keys())[0]])
    total_length = (total_length // block_size) * block_size
    result = {
        k: [t[i : i + block_size] for i in range(0, total_length, block_size)]
        for k, t in concatenated_examples.items()
    }
    result["labels"] = result["input_ids"].copy()
    return result

In [26]:
# Apply Grouping
print(f"\nGrouping texts into blocks of size {block_size} for language modeling...")
lm_dataset = tokenized_dataset.map(
    group_texts,
    batched=True
)
lm_dataset.set_format("torch")
print("Texts grouped for language modeling.")


Grouping texts into blocks of size 128 for language modeling...


  0%|          | 0/16 [00:00<?, ?ba/s]

  0%|          | 0/2 [00:00<?, ?ba/s]

  0%|          | 0/2 [00:00<?, ?ba/s]

Texts grouped for language modeling.


In [ ]:
# !pip uninstall -y transformers datasets huggingface_hub tokenizers

# !pip install transformers==4.25.0 datasets==2.5.0 huggingface_hub==0.10.0 tokenizers==0.12.1

In [38]:
from transformers import AutoModel
model = AutoModel.from_pretrained(local_bert_mini_dir).to(args.device)

Some weights of the model checkpoint at C:\Users\Pracioppo\Desktop\Peter DynAttn Proj\data\prajjwal1_bert_mini were not used when initializing BertModel: ['cls.seq_relationship.weight', 'cls.predictions.transform.dense.weight', 'cls.predictions.decoder.weight', 'cls.predictions.bias', 'cls.seq_relationship.bias', 'cls.predictions.decoder.bias', 'cls.predictions.transform.LayerNorm.weight', 'cls.predictions.transform.LayerNorm.bias', 'cls.predictions.transform.dense.bias']
- This IS expected if you are initializing BertModel from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing BertModel from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).


In [53]:
# Create DataLoaders
batch_size = 16
train_dataloader = DataLoader(lm_dataset["train"], shuffle=True, batch_size=batch_size)
eval_dataloader = DataLoader(lm_dataset["validation"], batch_size=batch_size)
test_dataloader = DataLoader(lm_dataset["test"], batch_size=batch_size)

for it, sample in enumerate(train_dataloader):
    print(f"\n--- Processing single batch (iteration {it}) ---")

    # 1. Extract input tensors from the 'sample' dictionary
    #    These are already PyTorch tensors due to lm_dataset.set_format("torch")
    input_ids = sample['input_ids']
    attention_mask = sample['attention_mask']
    token_type_ids = sample['token_type_ids'] # Essential for BERT-like models

    print(f"Shape of input_ids in this batch: {input_ids.shape}")
    print(f"Shape of attention_mask in this batch: {attention_mask.shape}")
    print(f"Shape of token_type_ids in this batch: {token_type_ids.shape}")

    # 2. Move these input tensors to the same device as your model (GPU or CPU)
    input_ids = input_ids.to(device)
    attention_mask = attention_mask.to(device)
    token_type_ids = token_type_ids.to(device)

    # 3. Pass inputs through the model to get embeddings
    with torch.no_grad(): # Disable gradient calculation for inference (saves memory, speeds up)
        model_output = model(
            input_ids=input_ids,
            attention_mask=attention_mask,
            token_type_ids=token_type_ids
        )
        
    # 4. Access the contextualized embeddings (the last hidden state)
    contextual_embeddings_for_this_batch = model_output.last_hidden_state
    
    break
    
with torch.no_grad(): # No need to compute gradients
    # This calls the forward method of the BertEmbeddings module
    # It takes input_ids, token_type_ids, and internally adds positional embeddings,
    # then applies LayerNorm and Dropout.
    initial_input_embeddings = model.embeddings(
        input_ids=input_ids,
        token_type_ids=token_type_ids # Pass token_type_ids for BERT models
    )
    
initial_input_embeddings.size()


--- Processing single batch (iteration 0) ---
Shape of input_ids in this batch: torch.Size([16, 128])
Shape of attention_mask in this batch: torch.Size([16, 128])
Shape of token_type_ids in this batch: torch.Size([16, 128])


torch.Size([16, 128, 256])

In [127]:
# def custom_similarity(query_emb, vocab_embs, alpha = 1):
#     """
#     Calculates custom similarity
#     """

#     diff = query_emb.unsqueeze(0) - vocab_embs
#     p = 1/(alpha + diff**2)
    
#     scores = torch.mean(p,axis=1)
    
#     return scores

In [130]:
def embedding_to_nearest_token(vocab_embed, query_embed):
    """
    Given an embedding, find the nearest embedding in the vocabulary and output the corresponding token.
    """

    similarities = F.cosine_similarity(query_embed.unsqueeze(0), vocab_embeddings, dim=1)

    # Find the top K similarities and their indices
    top_k_similarities, top_k_indices = torch.topk(similarities, k=1)

    # Get the token ID
    token_id = top_k_indices.item()
    # Convert the token ID back to a readable token string
    token_string = tokenizer.convert_ids_to_tokens(token_id)
    similarity_score = top_k_similarities.item()

    return token_string, similarity_score

query_embed = initial_input_embeddings[0,0]
vocab_embed = model.embeddings.word_embeddings.weight

token_string, similarity_score = embedding_to_nearest_token(vocab_embed, query_embed)

print(f"Token: '{token_string}', Similarity: {similarity_score:.4f}")

Token: '.', Similarity: 0.6112
